In [476]:
#Import the necessary libraries: pandas, matplotlib, and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [477]:
# Load CSV
df = pd.read_csv("Data.csv")

In [478]:
df.head()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación
0,BOLLERIA,VENTA,2017-01-01,2017,1,8,5820,3.0,4.091,12.273,Año Nuevo,Sí,Sunday,12.54,67.47,1.22,0.0
1,BOLLERIA,VENTA,2017-01-01,2017,1,9,3880,12.0,2.182,26.184,Año Nuevo,Sí,Sunday,12.54,67.47,1.22,0.0
2,BOLLERIA,VENTA,2017-01-01,2017,1,9,3894,6.0,3.000,18.000,Año Nuevo,Sí,Sunday,12.54,67.47,1.22,0.0
3,BOLLERIA,VENTA,2017-01-01,2017,1,9,3960,21.0,1.773,37.233,Año Nuevo,Sí,Sunday,12.54,67.47,1.22,0.0
4,BOLLERIA,VENTA,2017-01-01,2017,1,9,3974,15.0,3.273,49.095,Año Nuevo,Sí,Sunday,12.54,67.47,1.22,0.0


In [479]:
df.tail()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación
942458,PASTELERIA,VENTA,2023-05-18,2023,5,20,5619,6.00,4.364,26.1840,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942459,PASTELERIA,VENTA,2023-05-18,2023,5,20,7689,6.00,4.909,29.4540,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942460,PASTELERIA,VENTA,2023-05-18,2023,5,20,5730,3.00,4.364,13.0920,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942461,PASTELERIA,VENTA,2023-05-18,2023,5,20,5936,0.57,65.318,37.2313,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942462,PASTELERIA,MERMA,2023-05-18,2023,5,21,8877,6.00,8.727,52.3620,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0


In [480]:
# Get unique values ​​from each family
unique_articles = df.groupby('Familia')['Articulo'].unique().reset_index()

# Print results
print(unique_articles)

      Familia                                           Articulo
0    BOLLERIA  [5820, 3880, 3894, 3960, 3974, 5803, 5804, 626...
1   PANADERIA  [968, 854, 900, 936, 940, 970, 1043, 1084, 109...
2  PASTELERIA  [5619, 5837, 6549, 5403, 5498, 5518, 5629, 553...


In [481]:
# Get unique values ​​from 'Holiday' column
unique_festivo = df['Festivo'].unique()

# Print results
print(unique_festivo)

['Año Nuevo' 'Día no festivo' 'Día de Reyes' 'Día de San Valentín'
 'Día del Padre' 'Domingo de Ramos' 'Lunes Santo' 'Martes Santo'
 'Miércoles Santo' 'Jueves Santo' 'Viernes Santo' 'Sábado Santo'
 'Domingo de Resurrección' 'Día de la Cruz' 'Día de la Madre'
 'Día de San Juan' 'Día de la Virgen del Carmen' 'Feria de Málaga'
 'Día de la Victoria' 'Fiesta de la Virgen de la Victoria'
 'Día de la Hispanidad (Fiesta Nacional de España)' 'Noche de Halloween'
 'Día de Todos los Santos' 'Día de la Constitución' 'Nochebuena' 'Navidad'
 'Nochevieja']


In [482]:
# Convert 'Date' column to datetime type
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Define the new holidays and their dates until 2023-05-18
new_holidays = {
    'Día del Trabajador': ['2021-05-01', '2022-05-01', '2023-05-01'],
    'Día de la Inmaculada': ['2021-12-08', '2022-12-08']
}

# Update the 'Holiday' and 'It_Holiday' columns on the corresponding dates
for holiday, dates in new_holidays.items():
    for date in dates:
        df.loc[df['Fecha'] == date, 'Festivo'] = holiday
        df.loc[df['Fecha'] == date, 'Es_Festivo'] = 'Sí'

# Filter and print the rows corresponding to the new holidays
filtered_df = df[df['Fecha'].isin([pd.to_datetime(date) for dates in new_holidays.values() for date in dates])]

In [483]:
# Define the date range
start_date = '2021-05-01'
end_date = '2023-05-18'

# Filter the DataFrame to get the desired date range
df_filtrado = df[(df['Fecha'] >= start_date) & (df['Fecha'] <= end_date)]

# Filter the DataFrame to get only the rows where 'Type' is 'SALE'
df_filtrado = df_filtrado[df_filtrado['Tipo'] == 'VENTA']

In [484]:
# Print the filtered DataFrame
df_filtrado.head()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación
253314,BOLLERIA,VENTA,2021-05-01,2021,5,7,3960,12.0,2.318,27.816,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0
253315,BOLLERIA,VENTA,2021-05-01,2021,5,7,6626,3.0,2.727,8.181,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0
253316,BOLLERIA,VENTA,2021-05-01,2021,5,8,3880,6.0,2.591,15.546,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0
253317,BOLLERIA,VENTA,2021-05-01,2021,5,8,3960,9.0,2.318,20.862,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0
253318,BOLLERIA,VENTA,2021-05-01,2021,5,9,3880,15.0,2.591,38.865,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0


In [485]:
df_filtrado.tail()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación
942457,PASTELERIA,VENTA,2023-05-18,2023,5,20,5404,3.00,43.500,130.5000,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942458,PASTELERIA,VENTA,2023-05-18,2023,5,20,5619,6.00,4.364,26.1840,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942459,PASTELERIA,VENTA,2023-05-18,2023,5,20,7689,6.00,4.909,29.4540,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942460,PASTELERIA,VENTA,2023-05-18,2023,5,20,5730,3.00,4.364,13.0920,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0
942461,PASTELERIA,VENTA,2023-05-18,2023,5,20,5936,0.57,65.318,37.2313,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0


In [486]:
# Add 'rain' column
df_filtrado['lluvia'] = df_filtrado['Precipitación'].apply(lambda x: 'Sí' if x > 0 else 'No')

In [487]:
df_filtrado.head()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación,lluvia
253314,BOLLERIA,VENTA,2021-05-01,2021,5,7,3960,12.0,2.318,27.816,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0,No
253315,BOLLERIA,VENTA,2021-05-01,2021,5,7,6626,3.0,2.727,8.181,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0,No
253316,BOLLERIA,VENTA,2021-05-01,2021,5,8,3880,6.0,2.591,15.546,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0,No
253317,BOLLERIA,VENTA,2021-05-01,2021,5,8,3960,9.0,2.318,20.862,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0,No
253318,BOLLERIA,VENTA,2021-05-01,2021,5,9,3880,15.0,2.591,38.865,Día del Trabajador,Sí,Saturday,0.0,0.0,0.0,0.0,No


In [488]:
df_filtrado.tail()

,Familia,Tipo,Fecha,Año,Mes,Hora,Articulo,Cantidad,Precio,Importe,Festivo,Es_Festivo,Dia_semana,Temperatura,Humedad,Viento,Precipitación,lluvia
942457,PASTELERIA,VENTA,2023-05-18,2023,5,20,5404,3.00,43.500,130.5000,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0,No
942458,PASTELERIA,VENTA,2023-05-18,2023,5,20,5619,6.00,4.364,26.1840,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0,No
942459,PASTELERIA,VENTA,2023-05-18,2023,5,20,7689,6.00,4.909,29.4540,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0,No
942460,PASTELERIA,VENTA,2023-05-18,2023,5,20,5730,3.00,4.364,13.0920,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0,No
942461,PASTELERIA,VENTA,2023-05-18,2023,5,20,5936,0.57,65.318,37.2313,Día no festivo,No,Thursday,0.0,0.0,0.0,0.0,No


In [489]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335699 entries, 253314 to 942461
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Familia        335699 non-null  object        
 1   Tipo           335699 non-null  object        
 2   Fecha          335699 non-null  datetime64[ns]
 3   Año            335699 non-null  int64         
 4   Mes            335699 non-null  int64         
 5   Hora           335699 non-null  int64         
 6   Articulo       335699 non-null  int64         
 7   Cantidad       335699 non-null  float64       
 8   Precio         335699 non-null  float64       
 9   Importe        335699 non-null  float64       
 10  Festivo        335699 non-null  object        
 11  Es_Festivo     335699 non-null  object        
 12  Dia_semana     335699 non-null  object        
 13  Temperatura    335699 non-null  float64       
 14  Humedad        335699 non-null  float64       
 15  

In [490]:
# Convert Date column to datetime
df_filtrado["Fecha"] = pd.to_datetime(df_filtrado["Fecha"])

In [491]:
# Assign 'Date' as index of the DataFrame
df_filtrado.set_index('Fecha', inplace=True)

In [492]:
# Convert categorical columns to dummy variables without removing the first category
df_filtrado = pd.get_dummies(df_filtrado, columns=['Familia', 'Tipo', 'Dia_semana','Es_Festivo', 'lluvia', 'Festivo'], drop_first=False)

In [493]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 335699 entries, 2021-05-01 to 2023-05-18
Data columns (total 55 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Año                                                       335699 non-null  int64  
 1   Mes                                                       335699 non-null  int64  
 2   Hora                                                      335699 non-null  int64  
 3   Articulo                                                  335699 non-null  int64  
 4   Cantidad                                                  335699 non-null  float64
 5   Precio                                                    335699 non-null  float64
 6   Importe                                                   335699 non-null  float64
 7   Temperatura                                               335699 non-null  f

In [494]:
# Ensure consistency of features
variables_seleccionadas = [
    'Importe', 'Precio', 'Hora', 'Articulo', 'Humedad', 'Temperatura', 
    'Viento', 'Precipitación', 'Es_Festivo_Sí', 'Dia_semana_Sunday', 
    'Dia_semana_Monday', 'Dia_semana_Tuesday', 'Dia_semana_Wednesday', 
    'Dia_semana_Thursday', 'Dia_semana_Friday', 'Dia_semana_Saturday',
    'Año', 'Es_Festivo_No', 'Familia_BOLLERIA', 'Familia_PANADERIA', 'Familia_PASTELERIA'
]

In [495]:
# Add missing columns with default values
for col in variables_seleccionadas:
    if col not in data_prediccion.columns:
        data_prediccion[col] = 0

In [496]:
# List of specific products
productos_especificos = [1043, 1084, 3880, 3960, 417, 5403, 5404, 5803, 6286, 6425, 6451, 6523, 6549, 900, 968]

In [497]:
# Modeled by Product Family
resultados = {}
predicciones = {}

for familia_col in familia_columnas:
    data_familia = df_filtrado[df_filtrado[familia_col] == 1]
    if len(data_familia) > 0:
        articulos = data_familia['Articulo'].unique()
        for articulo in articulos:
            if articulo not in productos_especificos:
                continue
            data_articulo = data_familia[data_familia['Articulo'] == articulo]
            if len(data_articulo) < 5:
                print(f"Artículo {articulo} en familia {familia_col} tiene menos de 5 registros. Se imputará con la media de la familia.")
                media_familia = data_familia['Cantidad'].mean()
                data_articulo = data_articulo.copy()
                data_articulo.loc[:, 'Cantidad'] = media_familia
            X = data_articulo[variables_seleccionadas]
            y = data_articulo['Cantidad']
            if len(X) != len(y):
                continue
            if len(data_articulo) > 5:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                scaler_X = MinMaxScaler()
                scaler_y = MinMaxScaler()
                X_train_scaled = scaler_X.fit_transform(X_train)
                X_test_scaled = scaler_X.transform(X_test)
                y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
                y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()
                if X_train_scaled.shape[0] != y_train_scaled.shape[0]:
                    print(f"Error: Inconsistent number of samples in X_train_scaled and y_train_scaled for artículo {articulo} in familia {familia_col}.")
                    continue
                modelo = RandomForestRegressor(n_estimators=50, max_depth=5, min_samples_split=2)
                modelo.fit(X_train_scaled, y_train_scaled)
                y_pred_scaled = modelo.predict(X_test_scaled)
                y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
                mse = mean_squared_error(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                rmse = mean_squared_error(y_test, y_pred, squared=False)
                r2 = r2_score(y_test, y_pred)
            else:
                # Use the entire data set for training and testing
                scaler_X = MinMaxScaler()
                scaler_y = MinMaxScaler()
                X_scaled = scaler_X.fit_transform(X)
                y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
                modelo = RandomForestRegressor(n_estimators=50, max_depth=5, min_samples_split=2)
                modelo.fit(X_scaled, y_scaled)
                y_pred_scaled = modelo.predict(X_scaled)
                y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
                mse = mean_squared_error(y, y_pred)
                mae = mean_absolute_error(y, y_pred)
                rmse = mean_squared_error(y, y_pred, squared=False)
                r2 = r2_score(y, y_pred)
            familia = familia_col.split('_')[1]
            if familia not in resultados:
                resultados[familia] = {}
            resultados[familia]['RandomForest'] = {'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'R2': r2}
            y_pred_fechas_scaled = modelo.predict(scaler_X.transform(data_prediccion[variables_seleccionadas]))
            y_pred_fechas = scaler_y.inverse_transform(y_pred_fechas_scaled.reshape(-1, 1)).flatten()
            if 'RandomForest' not in predicciones:
                predicciones['RandomForest'] = []
            for fecha, pred in zip(fechas_prediccion, y_pred_fechas):
                predicciones['RandomForest'].append({'Fecha': fecha, 'Articulo': articulo, 'Familia': familia, 'Mejor_Modelo': 'RandomForest', 'Prediccion': pred})

In [498]:
# Save downscaled predictions to a CSV file
predicciones_df = pd.DataFrame(predicciones['RandomForest'])
predicciones_df.to_csv('predicciones_diarias.csv', index=False)

In [499]:
# Print performance metrics
for familia, metricas in resultados.items():
    print(f"Métricas de rendimiento para la familia {familia}:")
    for modelo, valores in metricas.items():
        print(f"  Modelo: {modelo}")
        print(f"    MSE: {valores['MSE']}")
        print(f"    MAE: {valores['MAE']}")
        print(f"    RMSE: {valores['RMSE']}")
        print(f"    R2: {valores['R2']}")
    print("\n")

print("Proceso completado.")

Métricas de rendimiento para la familia BOLLERIA:
  Modelo: RandomForest
    MSE: 0.11519104036481935
    MAE: 0.08229739280539516
    RMSE: 0.33939805592374767
    R2: 0.9978005610445926


Métricas de rendimiento para la familia PANADERIA:
  Modelo: RandomForest
    MSE: 0.07246883320091028
    MAE: 0.04374496711402644
    RMSE: 0.26920035884246196
    R2: 0.9979242382810142


Métricas de rendimiento para la familia PASTELERIA:
  Modelo: RandomForest
    MSE: 1.3004539682539678
    MAE: 0.07396825396825431
    RMSE: 1.1403744859711513
    R2: 0.8530875742487072


Proceso completado.
